In [12]:
from pyspark.sql import SparkSession

In [13]:
import os
print(os.environ.get("HUDI_SPARK_BUNDLE"))

HUDI_SPARK_BUNDLE = os.environ.get("HUDI_SPARK_BUNDLE")

/var/hoodie/ws/docker/hoodie/hadoop/hive_base/target/hoodie-spark-bundle.jar


In [14]:
spark = SparkSession.builder.appName('HudiCRUD').config("spark.jars", HUDI_SPARK_BUNDLE).config("spark.serializer", "org.apache.spark.serializer.KryoSerializer").config("spark.sql.extensions", "org.apache.spark.sql.hudi.HoodieSparkSessionExtension").config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.hudi.catalog.HoodieCatalog").config("spark.sql.hive.convertMetastoreParquet", "false").getOrCreate()

In [15]:
columns = ["id","creation_date", "status", "last_update_time"]
data = [
    ("100", "2015-01-01", "unprocessed", "2015-01-01T13:51:39.340396Z"),
    ("101", "2015-01-01", "unprocessed", "2015-01-01T12:14:58.597216Z"),
    ("102", "2015-01-01", "unprocessed", "2015-01-01T13:51:40.417052Z"),
    ("103", "2015-01-01", "unprocessed", "2015-01-01T13:51:40.519832Z"),
    ("104", "2015-01-02", "unprocessed", "2015-01-01T12:15:00.512679Z"),
    ("105", "2015-01-02", "unprocessed", "2015-01-01T13:51:42.248818Z")
]

In [16]:
inputDF = spark.createDataFrame(data).toDF(*columns)
inputDF.show(truncate = False)

+---+-------------+-----------+---------------------------+
|id |creation_date|status     |last_update_time           |
+---+-------------+-----------+---------------------------+
|100|2015-01-01   |unprocessed|2015-01-01T13:51:39.340396Z|
|101|2015-01-01   |unprocessed|2015-01-01T12:14:58.597216Z|
|102|2015-01-01   |unprocessed|2015-01-01T13:51:40.417052Z|
|103|2015-01-01   |unprocessed|2015-01-01T13:51:40.519832Z|
|104|2015-01-02   |unprocessed|2015-01-01T12:15:00.512679Z|
|105|2015-01-02   |unprocessed|2015-01-01T13:51:42.248818Z|
+---+-------------+-----------+---------------------------+



In [17]:
!echo $HUDI_SPARK_BUNDLE

/var/hoodie/ws/docker/hoodie/hadoop/hive_base/target/hoodie-spark-bundle.jar


In [1]:
table_name = "ztest_hudi_table_cow"
base_path = f"hdfs:///tmp/{table_name}"

# HUDI Configuration

In [19]:
hudi_conf = {
    "hoodie.table.name": table_name,
    "hoodie.datasource.write.recordkey.field": "id",
    "hoodie.datasource.write.storage.type": "COPY_ON_WRITE",
    "hoodie.datasource.write.partitionpath.field": "creation_date",
    "hoodie.datasource.write.precombine.field": "last_update_time",
    "hoodie.datasource.write.operation": "insert",
    "hoodie.write.markers.type": "DIRECT",
    "hoodie.datasource.write.hive_style_partitioning": "true"
}

In [20]:
# Write the DataFrame to a Hudi COW table
inputDF.write \
    .format("hudi") \
    .options(**hudi_conf) \
    .mode("overwrite") \
    .save(base_path)

25/08/11 17:32:16 WARN DataSourceOptionsHelper$: hoodie.datasource.write.storage.type is deprecated and will be removed in a later release; Please use hoodie.datasource.write.table.type
25/08/11 17:32:16 WARN DataSourceOptionsHelper$: hoodie.datasource.write.storage.type is deprecated and will be removed in a later release; Please use hoodie.datasource.write.table.type
25/08/11 17:32:24 WARN HoodieStorage: Fail to rename hdfs:/tmp/ztest_hudi_table_cow/.hoodie/metadata/column_stats/.hoodie_partition_metadata.66cefcb9-b589-46cf-b30c-1b2bea5550ef to hdfs:/tmp/ztest_hudi_table_cow/.hoodie/metadata/column_stats/.hoodie_partition_metadata, target file exists: true
                                                                                

In [21]:
spark.read.format("hudi").load(base_path).show(truncate = False)

+-------------------+---------------------+------------------+------------------------+-------------------------------------------------------------------------+---+-------------+-----------+---------------------------+
|_hoodie_commit_time|_hoodie_commit_seqno |_hoodie_record_key|_hoodie_partition_path  |_hoodie_file_name                                                        |id |creation_date|status     |last_update_time           |
+-------------------+---------------------+------------------+------------------------+-------------------------------------------------------------------------+---+-------------+-----------+---------------------------+
|20250811173216666  |20250811173216666_0_0|100               |creation_date=2015-01-01|af4fcb4f-856f-4155-beb3-588961cf5310-0_0-65-151_20250811173216666.parquet|100|2015-01-01   |unprocessed|2015-01-01T13:51:39.340396Z|
|20250811173216666  |20250811173216666_0_1|101               |creation_date=2015-01-01|af4fcb4f-856f-4155-beb3-588961cf5